In [12]:
import geopandas as gpd
import matplotlib.axes
from shapely.geometry import LineString
import requests
import io
import zipfile
import os
from bs4 import BeautifulSoup
import pandas as pd
from dbfread import DBF

In [3]:
from tqdm.notebook import tqdm

In [34]:
# download data from url
code_list = list(range(42001, 42134, 2))
sum_list = []
total_penn=[]
result_list = [[0 for i in range(4)] for j in range(67)]
i=0
for code in tqdm(code_list):
    url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/tl_2017_{}_roads.zip".format(code)
    name = "tl_2017_{}_roads.zip".format(code)
    r = requests.get(url)
    with open("./zips/{filename}".format(filename=name), "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile("./zips/{filename}".format(filename=name), 'r') as zip_ref:
        zip_ref.extractall("./tmp")
    shapefile = gpd.read_file('tmp/tl_2017_{}_roads.shp'.format(code))
    new_geometry = shapefile['geometry'].to_crs(epsg=32129)
    projection_length=new_geometry.length
    new_df=pd.concat([shapefile, projection_length],axis=1)
    new_df.rename(columns={0:'projction_length'},inplace=True)
    len1=sum(new_df.loc[new_df['MTFCC'] == 'S1100']['projction_length'])/1690 # Sum the length of primary roads
    len2=sum(new_df.loc[new_df['MTFCC'] == 'S1200']['projction_length'])/1690 # Sum the length of secondary roads
    len3=sum(new_df.loc[new_df['MTFCC'] == 'S1400']['projction_length'])/1690 # Sum the length of tertiary/local roads
    #with units of miles
    #print(len1,len2,len3)
    sum_length=len1+len2+len3
    result_list[i] = [len1, len2, len3, sum_length]
    i+=1

In [35]:
result_list[0:5]

[[0.0, 456.99080440932795, 1552.9559983109743, 2009.9468027203022],
 [472.3541071338303, 942.7436903555109, 6989.645046575266, 8404.742844064607],
 [0.0, 361.1772563644404, 3163.439580954565, 3524.6168373190053],
 [154.80092122642992,
  487.4866931548235,
  1914.5564489875287,
  2556.8440633687824],
 [270.4191530808973, 557.6226991775884, 5670.428980567093, 6498.470832825579]]

In [36]:
result_df = pd.DataFrame(result_list, columns=['Primary_length', 'Secondary_length', 'Local_length', 'Total_length'])

In [37]:
result_df

,Primary_length,Secondary_length,Local_length,Total_length
0,0.000000,456.990804,1552.955998,2009.946803
1,472.354107,942.743690,6989.645047,8404.742844
2,0.000000,361.177256,3163.439581,3524.616837
3,154.800921,487.486693,1914.556449,2556.844063
4,270.419153,557.622699,5670.428981,6498.470833
...,...,...,...,...
62,233.160618,672.107397,3642.023610,4547.291625
63,12.444536,530.310016,2772.328350,3315.082903
64,257.427540,1001.857772,5072.090346,6331.375659
65,0.000000,170.575749,1390.058933,1560.634682


In [41]:
pa_county=pd.read_csv('PA_county.csv')

In [43]:
pa=pd.concat([pa_county, result_df],axis=1)

In [44]:
pa

,County_code,County_Name,Primary_length,Secondary_length,Local_length,Total_length
0,42001,Adams,0.000000,456.990804,1552.955998,2009.946803
1,42003,Allegheny,472.354107,942.743690,6989.645047,8404.742844
2,42005,Armstrong,0.000000,361.177256,3163.439581,3524.616837
3,42007,Beaver,154.800921,487.486693,1914.556449,2556.844063
4,42009,Bedford,270.419153,557.622699,5670.428981,6498.470833
...,...,...,...,...,...,...
62,42125,Washington,233.160618,672.107397,3642.023610,4547.291625
63,42127,Wayne,12.444536,530.310016,2772.328350,3315.082903
64,42129,Westmoreland,257.427540,1001.857772,5072.090346,6331.375659
65,42131,Wyoming,0.000000,170.575749,1390.058933,1560.634682


In [45]:
pa.to_csv('pa_tiger.csv')

#### EPSG (two assumptions )

In [ ]:
# Part 1 searching correponidng plane
# result for Alleghney: PA011(BERKS CO.) NAD83 ZONE = 3702 (LAMBERT|SOUTHERN ZONE)

# Part 2 finding corrsponding EPSG

# EPSG:2271: NAD83 / Pennsylvania North (ftUS)
# EPSG:2272: NAD83 / Pennsylvania South (ftUS)

#1 EPSG:3362: NAD83(HARN) / Pennsylvania North 
# EPSG:3363: NAD83(HARN) / Pennsylvania North (ftUS)
# EPSG:3364: NAD83(HARN) / Pennsylvania South
# EPSG:3365: NAD83(HARN) / Pennsylvania South (ftUS)

#2 EPSG:3649: NAD83(NSRS2007) / Pennsylvania North
# EPSG:3650: NAD83(NSRS2007) / Pennsylvania North (ftUS)
# EPSG:3651: NAD83(NSRS2007) / Pennsylvania South
# EPSG:3652: NAD83(NSRS2007) / Pennsylvania South (ftUS)

#3 EPSG:32128: NAD83 / Pennsylvania North
# EPSG:32129: NAD83 / Pennsylvania South

# EPSG:32028: NAD27 / Pennsylvania North
# EPSG:32029: NAD27 / Pennsylvania South

# NAD27 NAD83 are different datum plane. 
# First assumption: I open Allegheny seraching for corresponding state plane coordination 
# reference system, if we choose NAD83, all counties in PA is using NAD83.

# For most of the United States, the differences between HARN coordinates and NSRS2007 are a few centimeters. After 
# calculate #1 2 3 epsg, having similiar results. While 2271 not.

# north is 399610139, south is 399589079
# Second assumption, south and north are very close for the total number, so choose one plane for each state.

### Whole State Penn

#### All counties in Penn using EPSG:32129: NAD83 / Pennsylvania South

In [5]:
from tqdm.notebook import tqdm

In [ ]:
# download data from url
code_list = list(range(42001, 42134, 2))
sum_list = []
total_penn=[]
for code in tqdm(code_list):
    url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/tl_2017_{}_roads.zip".format(code)
    name = "tl_2017_{}_roads.zip".format(code)
    r = requests.get(url)
    with open("./zips/{filename}".format(filename=name), "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile("./zips/{filename}".format(filename=name), 'r') as zip_ref:
        zip_ref.extractall("./tmp")
    shapefile2 = gpd.read_file('tmp/tl_2017_{}_roads.shp'.format(code))
    sum_ = shapefile2['geometry'].to_crs(epsg=32129)

    sum_list.append(sum_)
    #print("error in ", code)

In [36]:
sum_list2 = []
for item in sum_list:
    s = sum(item.length)
    sum_list2.append(s)

In [30]:
total_length=sum(sum_list2)
print(total_length)
print(total_length/1609)
#with units of meters
# public roads total number for penn:  120,521 miles
# state highway agency owned roads for penn: 39,733 miles

399589079.400613
75679.74988647974


#### All counties in Penn using EPSG:32128: NAD83 / Pennsylvania North 

In [15]:
from tqdm import tqdm

In [8]:
code_list = list(range(42001, 42134, 2))
sum_list = []
for code in tqdm(code_list):
    try:
        url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/tl_2017_{}_roads.zip".format(code)
        r = requests.get(url)
        with open("tmp.zip", "wb") as f:
            f.write(r.content)
        with zipfile.ZipFile("tmp.zip", 'r') as zip_ref:
            zip_ref.extractall("./tmp")
        shapefile2 = gpd.read_file('tmp/tl_2017_{}_roads.shp'.format(code))
        sum_ = shapefile2['geometry'].to_crs(epsg=32128)
        sum_list.append(sum_)
        
    except:
        print("error in ", code)
        continue

error in  42037
error in  42079
error in  42083
error in  42085
error in  42089
error in  42091
error in  42093
error in  42101
error in  42103
error in  42105
error in  42115
error in  42119



In [ ]:
sum_list2 = list(map(lambda x : sum(x.length), sum_list))

In [33]:
print(sum(sum_list2))
#print(sum(sum_list2)/5280)
print(sum(sum_list2)/1609)
# using all north is 75683, using all south is 75679 aftering the total number/5280
# EPSG:32128: NAD83 / Pennsylvania North: 399610139meters, 248359miles
# EPSG:2271: NAD83 / Pennsylvania North (ftUS): 1311054266.3281152feet, 248305miles
# public roads total number for penn:  120,521 miles in highway statistics
# state highway agency owned roads for penn: 39,733 miles

399610139.59708816
248359.3160951449


In [ ]:
# north is 399610139, south is 399589079
# Second assumption, south and north are very close for the total number, so choose one plane for each state.

### AL

In [17]:
from tqdm.notebook import tqdm

In [25]:
code_list = list(range(1001, 1134, 2))
sum_list = []
for code in tqdm(code_list):
    url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/tl_2017_0{}_roads.zip".format(code)
    r = requests.get(url)
    with open("tmp.zip", "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile("tmp.zip", 'r') as zip_ref:
        zip_ref.extractall("./tmp")
    shapefile2 = gpd.read_file('tmp/tl_2017_0{}_roads.shp'.format(code))
    sum_ = shapefile2['geometry'].to_crs(epsg=3465)
    sum_list.append(sum_)

In [26]:
sum_list2 = list(map(lambda x : sum(x.length), sum_list))
print(sum_list2)

[3016210.752405435, 8761965.956803164, 2813997.670766218, 2914330.9160377947, 3411528.2707341276, 2152911.4321498405, 3540340.4823145797, 5089106.271556616, 3482841.8730705245, 3676156.509042458, 3216687.7541687232, 2836227.9700234146, 3864632.372294832, 2566556.5452854536, 2306190.903273507, 3136704.840537609, 3392426.6522158165, 3689758.762913684, 2658210.5571793425, 4951794.261777457, 2514216.3652911363, 4626164.90420539, 3258177.674724915, 4972454.726227399, 5154625.510927897, 3703789.909873388, 4514489.660617159, 4055720.975668797, 2416521.549406045, 3036674.742962581, 2854505.3482353054, 2403096.228597298, 1919081.4863162818, 2873230.482723203, 4029907.7834775345, 4367548.699738663, 13819767.180958465, 2645209.5830626604, 5633006.740459391, 2762739.6197544755, 5620643.563884814, 4432212.820579137, 2547720.103397979, 2354291.024925217, 8927076.639584802, 3210982.840601678, 3699639.8732775613, 3945464.9610439595, 13502520.46844365, 3348952.792613378, 5468155.862100597, 3738912.4353

In [28]:
print(sum(sum_list2))
print(sum(sum_list2)/5280)
#print(sum(sum_list2)/1609)

270641700.1275958
51257.89775143859


### AZ

In [34]:
code_list = list(range(4001, 4028, 2))
sum_list = []
for code in tqdm(code_list):
    try:
        url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/tl_2017_0{}_roads.zip".format(code)
        r = requests.get(url)
        with open("tmp.zip", "wb") as f:
            f.write(r.content)
        with zipfile.ZipFile("tmp.zip", 'r') as zip_ref:
            zip_ref.extractall("./tmp")
        shapefile2 = gpd.read_file('tmp/tl_2017_0{}_roads.shp'.format(code))
        sum_ = shapefile2['geometry'].to_crs(epsg=3478)
        sum_list.append(sum_)
    except:
        print("error in ", code)
        continue

In [35]:
sum_list2 = list(map(lambda x : sum(x.length), sum_list))
print(sum_list2)

[40342537.65631483, 16167783.257124448, 45141670.34126591, 9255645.92319104, 7322604.65231582, 2370899.8101191586, 52128254.51816357, 29898834.93516911, 31016932.84110085, 25141672.234236147, 19683875.284564964, 4228235.168717401, 22477126.362081073, 10012745.52768065]


In [38]:
print(sum(sum_list2))
print(sum(sum_list2)/5280)

315188818.5120449
59694.851990917596


### VMT data

In [60]:
vmt=pd.read_csv("countyVMT groupby 6-30-sum-daily.csv", skiprows=0)
vmt = vmt.drop(index=[0, 1], columns=['Unnamed: 1', 'VIN'])

In [62]:
vmt = vmt.set_index("Unnamed: 0")

In [74]:
vmt = vmt.applymap(lambda x: int(float(x)))

In [75]:
vmt

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2009 daily,2010 daily,2011 daily,2012 daily,2013 daily,2014 daily,2015 daily,2016 daily,2017 daily,2018 daily
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
ADAMS,0,0,0,0,26584,56242,80600,586197,1572742,2559467,...,7012,11468,11968,13109,14979,15457,13008,9727,880,207
ALLEGHENY,1737697798,2122643338,2353730136,2160414664,2016237720,1972750712,1966638964,2030300146,2072517121,2162644661,...,5925053,6059461,6134040,6319265,6524467,6870422,6366980,5483103,1660562,388868
ARMSTRONG,32429,38415,43291,26951,34170,44176,47571,56512,143232,160475,...,439,677,711,1375,2636,3681,3071,1894,229,0
BEAVER,234893603,288729126,315325989,278458251,257164457,253589278,255003183,256185291,253029414,255477330,...,699937,687825,664765,650525,642233,648686,596007,506034,138075,31720
BEDFORD,0,0,0,0,1657,8388,7890,6997,174201,236422,...,647,815,681,2405,11435,14453,14394,11511,674,338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WASHINGTON,274217967,336378262,369911022,328311885,302343566,301787191,305615335,318298297,326064498,346444056,...,949161,975036,983703,1006223,1038373,1103285,1036584,895339,301935,89872
WAYNE,0,0,0,0,0,0,0,341426,886095,1451534,...,3976,4925,6153,8116,11467,15399,15207,13397,2085,1029
WESTMORELAND,492123552,605128681,665477895,601499024,564038635,558144336,558376630,576052808,581416770,598314102,...,1639216,1635277,1623813,1645121,1686512,1775762,1650733,1421288,377790,72081


In [83]:
print(vmt[0:35])

                  2000        2001        2002        2003        2004  \
Unnamed: 0                                                               
ADAMS                0           0           0           0       26584   
ALLEGHENY   1737697798  2122643338  2353730136  2160414664  2016237720   
ARMSTRONG        32429       38415       43291       26951       34170   
BEAVER       234893603   288729126   315325989   278458251   257164457   
BEDFORD              0           0           0           0        1657   
BERKS            76334       92504       94507      252671   502068576   
BLAIR                0           0           0           0   169926470   
BRADFORD             0           0           0           0           0   
BUCKS        824715824  1040583123  1177039888  1104320082  1078112258   
BUTLER           25046       23497       24705       10747        9683   
CAMBRIA              0           0           0        6592   192420820   
CAMERON              0           0    

In [99]:
#vmt.columns.values.tolist()

In [90]:
new_vmt1=vmt['2017 daily']

In [91]:
new_vmt1

Unnamed: 0
ADAMS               880
ALLEGHENY       1660562
ARMSTRONG           229
BEAVER           138075
BEDFORD             674
                 ...   
WASHINGTON       301935
WAYNE              2085
WESTMORELAND     377790
WYOMING             295
YORK             397417
Name: 2017 daily, Length: 74, dtype: int64

In [92]:
new_vmt2=vmt['2015 daily']

In [96]:
new=pd.concat([new_vmt1,new_vmt2],axis=1) 

In [98]:
new.to_csv('new.csv')

## All counties road length calcu

In [13]:
# web scrape
import re, requests
r = requests.get("https://www2.census.gov/geo/tiger/TIGER2017/ROADS/")
f = r.content.decode()
pattern = '</td><td><a href="(.*?)">(.*?)</a></td><td align="right">'
all_zips = re.findall(pattern, f)
all_zips = [item[0] for item in all_zips]

In [14]:
len(all_zips)

3231

In [ ]:
sum_list = []
total_penn=[]
result_list = {}

In [43]:
# download data from url
import os
crs = gpd.read_file('tl_2017_42003_roads/tl_2017_42003_roads.shp').crs
for code in tqdm(all_zips):
    try:
        if code in result_list:
            continue
        url = "https://www2.census.gov/geo/tiger/TIGER2017/ROADS/{}".format(code)
        name = "{}".format(code)
        r = requests.get(url)
        zip_file = zipfile.ZipFile(io.BytesIO(r.content))
        zip_names = zip_file.namelist()  # Get names of all the files in the folder

        shp = zip_file.extract(zip_names[3])  # .shp, .dbf and .shx must be downloaded together to the local disk
        dbf = zip_file.extract(zip_names[1])
        shx = zip_file.extract(zip_names[-1])
        zip_file.close()
        shapefile = gpd.read_file(shp)  # Read the shapefile
        shapefile.crs = crs
        new_geometry = shapefile['geometry'].to_crs(epsg=32129)
        projection_length=new_geometry.length
        new_df=pd.concat([shapefile, projection_length],axis=1)
        new_df.rename(columns={0:'projction_length'},inplace=True)
        len1=sum(new_df.loc[new_df['MTFCC'] == 'S1100']['projction_length'])/1690 # Sum the length of primary roads
        len2=sum(new_df.loc[new_df['MTFCC'] == 'S1200']['projction_length'])/1690 # Sum the length of secondary roads
        len3=sum(new_df.loc[new_df['MTFCC'] == 'S1400']['projction_length'])/1690 # Sum the length of tertiary/local roads
        #with units of miles
        #print(len1,len2,len3)
        sum_length=len1+len2+len3
        k = [len1, len2, len3, sum_length]
        result_list[code] = k
        os.remove(shp)
        os.remove(dbf)
        os.remove(shx)
        print("Success in ", code)
    except Exception as e:
        print(e)
        print("Error in ", code)

Success in  tl_2017_48479_roads.zip
Success in  tl_2017_48481_roads.zip
Success in  tl_2017_48483_roads.zip
Success in  tl_2017_48485_roads.zip
Success in  tl_2017_48487_roads.zip
Success in  tl_2017_48489_roads.zip
Success in  tl_2017_48491_roads.zip
Success in  tl_2017_48493_roads.zip
Success in  tl_2017_48495_roads.zip
Success in  tl_2017_48497_roads.zip
Success in  tl_2017_48499_roads.zip
Success in  tl_2017_48501_roads.zip
Success in  tl_2017_48503_roads.zip
Success in  tl_2017_48505_roads.zip
Success in  tl_2017_48507_roads.zip
Success in  tl_2017_49001_roads.zip
Success in  tl_2017_49003_roads.zip
Success in  tl_2017_49005_roads.zip
Success in  tl_2017_49007_roads.zip
Success in  tl_2017_49009_roads.zip
Success in  tl_2017_49011_roads.zip
Success in  tl_2017_49013_roads.zip
Success in  tl_2017_49015_roads.zip
Success in  tl_2017_49017_roads.zip
Success in  tl_2017_49019_roads.zip
Success in  tl_2017_49021_roads.zip
Success in  tl_2017_49023_roads.zip
Success in  tl_2017_49025_ro

Success in  tl_2017_53075_roads.zip
Success in  tl_2017_53077_roads.zip
Success in  tl_2017_54001_roads.zip
Success in  tl_2017_54003_roads.zip
Success in  tl_2017_54005_roads.zip
Success in  tl_2017_54007_roads.zip
Success in  tl_2017_54009_roads.zip
Success in  tl_2017_54011_roads.zip
Success in  tl_2017_54013_roads.zip
Success in  tl_2017_54015_roads.zip
Success in  tl_2017_54017_roads.zip
Success in  tl_2017_54019_roads.zip
Success in  tl_2017_54021_roads.zip
Success in  tl_2017_54023_roads.zip
Success in  tl_2017_54025_roads.zip
Success in  tl_2017_54027_roads.zip
Success in  tl_2017_54029_roads.zip
Success in  tl_2017_54031_roads.zip
Success in  tl_2017_54033_roads.zip
Success in  tl_2017_54035_roads.zip
Success in  tl_2017_54037_roads.zip
Success in  tl_2017_54039_roads.zip
Success in  tl_2017_54041_roads.zip
Success in  tl_2017_54043_roads.zip
Success in  tl_2017_54045_roads.zip
Success in  tl_2017_54047_roads.zip
Success in  tl_2017_54049_roads.zip
Success in  tl_2017_54051_ro

Success in  tl_2017_72135_roads.zip
Success in  tl_2017_72137_roads.zip
Success in  tl_2017_72139_roads.zip
Success in  tl_2017_72141_roads.zip
Success in  tl_2017_72143_roads.zip
Success in  tl_2017_72145_roads.zip
Success in  tl_2017_72147_roads.zip
Success in  tl_2017_72149_roads.zip
Success in  tl_2017_72151_roads.zip
Success in  tl_2017_72153_roads.zip
Success in  tl_2017_78010_roads.zip
Success in  tl_2017_78020_roads.zip
Success in  tl_2017_78030_roads.zip



In [44]:
result_list

{'tl_2017_01001_roads.zip': [31.54907087139926,
  184.07804815923893,
  1577.388995417964,
  1793.0161144486021],
 'tl_2017_01003_roads.zip': [115.0300379950217,
  767.4726040448475,
  4190.016127180825,
  5072.518769220694],
 'tl_2017_01005_roads.zip': [0.0,
  375.2259132431614,
  1253.1117065517728,
  1628.3376197949342],
 'tl_2017_01007_roads.zip': [0.0,
  262.3562717071134,
  1400.8204882425573,
  1663.1767599496707],
 'tl_2017_01009_roads.zip': [23.194005321834627,
  271.93592393921347,
  1597.8908065780552,
  1893.0207358391033],
 'tl_2017_01011_roads.zip': [0.0,
  224.41503212713627,
  812.6324395606492,
  1037.0474716877854],
 'tl_2017_01013_roads.zip': [66.13161625095502,
  239.6854279968429,
  1710.6156275181875,
  2016.4326717659853],
 'tl_2017_01015_roads.zip': [21.55320083086697,
  368.67466170089875,
  2514.5108453555413,
  2904.738707887307],
 'tl_2017_01017_roads.zip': [20.260775156642072,
  178.76622935345884,
  1802.2004713704623,
  2001.2274758805631],
 'tl_2017_0101

In [ ]:
import pickle

In [41]:
with open("result_list", "wb") as f:
    pickle.dump(result_list, f)

In [45]:
lst=list(result_list.values())

In [46]:
lst

[[31.54907087139926,
  184.07804815923893,
  1577.388995417964,
  1793.0161144486021],
 [115.0300379950217, 767.4726040448475, 4190.016127180825, 5072.518769220694],
 [0.0, 375.2259132431614, 1253.1117065517728, 1628.3376197949342],
 [0.0, 262.3562717071134, 1400.8204882425573, 1663.1767599496707],
 [23.194005321834627,
  271.93592393921347,
  1597.8908065780552,
  1893.0207358391033],
 [0.0, 224.41503212713627, 812.6324395606492, 1037.0474716877854],
 [66.13161625095502,
  239.6854279968429,
  1710.6156275181875,
  2016.4326717659853],
 [21.55320083086697,
  368.67466170089875,
  2514.5108453555413,
  2904.738707887307],
 [20.260775156642072,
  178.76622935345884,
  1802.2004713704623,
  2001.2274758805631],
 [0.0, 231.03687705026903, 1718.6197799956408, 1949.65665704591],
 [53.646042979644115,
  224.65347725592167,
  1408.3096066755334,
  1686.609126911099],
 [0.0, 157.60831637252983, 1435.2784521675774, 1592.8867685401071],
 [0.0, 273.7003771496954, 1801.750872601353, 2075.451249751

In [48]:
my_df = pd.DataFrame.from_dict(result_list, orient='index')

In [49]:
my_df

,0,1,2,3
tl_2017_01001_roads.zip,31.549071,184.078048,1577.388995,1793.016114
tl_2017_01003_roads.zip,115.030038,767.472604,4190.016127,5072.518769
tl_2017_01005_roads.zip,0.000000,375.225913,1253.111707,1628.337620
tl_2017_01007_roads.zip,0.000000,262.356272,1400.820488,1663.176760
tl_2017_01009_roads.zip,23.194005,271.935924,1597.890807,1893.020736
...,...,...,...,...
tl_2017_72151_roads.zip,13.680721,59.913180,232.069591,305.663493
tl_2017_72153_roads.zip,0.000000,68.180387,343.031530,411.211918
tl_2017_78010_roads.zip,0.000000,347.178642,644.634192,991.812833
tl_2017_78020_roads.zip,0.000000,59.978771,105.801974,165.780745


In [54]:
primary_length=sum(my_df[0])
print(primary_length)
secondary_length=sum(my_df[1])
print(secondary_length)
local_length=sum(my_df[2])
print(local_length)
total_length=sum(my_df[3])
print(total_length)

143062.21070567635
869174.6698698797
6461391.103729003
7473627.984304546
